In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('cleaned_amazon_data.csv')
# print(data.head())
print(data.isnull().sum())

ID             0
ProdID         0
Rating         0
ReviewCount    0
Category       0
Name           0
ImageURL       0
Description    0
Tags           0
dtype: int64


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['ReviewCount'] = scaler.fit_transform(data[['ReviewCount']])

In [7]:
print(data['ReviewCount'].max())

1.0


In [9]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 5))  # Assuming ratings are between 1 and 5
interaction_data = Dataset.load_from_df(data[['ID', 'ProdID', 'Rating']], reader)

# Split into training and testing sets
trainset, testset = train_test_split(interaction_data, test_size=0.2)

# Train a matrix factorization model (SVD)
model = SVD()
model.fit(trainset)

# Evaluate the model
predictions = model.test(testset)
print("RMSE:", rmse(predictions))

# Generate recommendations for a user
def recommend_for_user(user_id, model, data, top_n=5):
    user_products = data[data['ID'] == user_id]['ProdID'].unique()
    all_products = data['ProdID'].unique()
    recommendations = []

    for prod_id in all_products:
        if prod_id not in user_products:
            pred = model.predict(user_id, prod_id).est
            recommendations.append((prod_id, pred))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

# Recommend products for a specific user
user_id = 69  # Replace with an actual user ID
recommendations = recommend_for_user(user_id, model, data)
print("Recommendations:", recommendations)


RMSE: 0.2674
RMSE: 0.26740087332008317
Recommendations: [(698, 4.446715292404889), (1277, 4.431163928789429), (684, 4.42985879415289), (336, 4.399867998437867), (816, 4.396558722623985)]
